In [1]:
import os

export_root = '/data1/chence/single_view/Exported'
json_files = sorted([i for i in os.listdir(export_root) if i.endswith('.json')])

In [2]:
import json

merged_data = {}
for idx in range(len(json_files)):
    json_file = json_files[idx]
    with open(os.path.join(export_root, json_file), 'r') as f:
        data = json.load(f)
        merged_data.update(data)
print(len(merged_data))

89721


In [3]:
exportjpg_root = '/data1/chence/single_view/ExportedJPG'
os.makedirs(exportjpg_root, exist_ok=True)

with open(os.path.join(exportjpg_root, 'dataset.json'), 'w') as f:
    json.dump(merged_data, f)

In [8]:
os.system('cp -r %s %s' % (os.path.join(export_root, 'align_parsing'), exportjpg_root))

0

In [9]:
os.system('cp -r %s %s' % (os.path.join(export_root, 'head_parsing'), exportjpg_root))

0

In [10]:
import cv2

def work(image_name, export_root, exportjpg_root):
    try:
        base_folder, base_name = image_name.split('/')
        align_png_path = os.path.join(export_root, 'align_images', base_folder, base_name)
        assert os.path.exists(align_png_path)
        head_png_path = os.path.join(export_root, 'head_images', base_folder, base_name)
        assert os.path.exists(head_png_path)
        os.makedirs(os.path.join(exportjpg_root, 'align_images', base_folder), exist_ok=True)
        os.makedirs(os.path.join(exportjpg_root, 'head_images', base_folder), exist_ok=True)
        align_jpg_path = os.path.join(exportjpg_root, 'align_images', base_folder, base_name.replace('.png', '.jpg'))
        head_jpg_path = os.path.join(exportjpg_root, 'head_images', base_folder, base_name.replace('.png', '.jpg'))
        cv2.imwrite(align_jpg_path, cv2.imread(align_png_path))
        cv2.imwrite(head_jpg_path, cv2.imread(head_png_path))
    except Exception as e:
        print(e)

In [11]:
image_names = sorted(merged_data.keys())

In [12]:
import multiprocessing as mp
from tqdm import tqdm

global progress_bar
progress_bar = tqdm(total=len(image_names))

def update_progress_bar(_):
    progress_bar.update()

pool = mp.Pool(processes=64)
for image_name in image_names:
    pool.apply_async(work, (
        image_name,
        export_root,
        exportjpg_root,
    ), callback=update_progress_bar)
pool.close()
pool.join()

  0%|          | 0/89721 [00:00<?, ?it/s]

100%|█████████▉| 89718/89721 [48:41<00:00, 63.66it/s]  

100%|██████████| 89721/89721 [49:00<00:00, 63.66it/s]